In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
import glob
from skimage import color
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score
from sklearn import svm
from sklearn.model_selection import train_test_split
import joblib
import import_ipynb

from ipynb.fs.full.hog_config import hog_human, WIN_SIZE
# from ipynb.fs.full.nms import nmsf

In [2]:
clf = joblib.load('model_hog_svm.pkl')

In [3]:
def pyramid(im, scale_step=1.2, min_size=(40, 40)):
    yield im
    while True:
        h = int(im.shape[0] / scale_step)
        w = int(im.shape[1] / scale_step)
        if h < min_size[0] or w < min_size[1]:
            break
        im = cv2.resize(im, (w, h))
        yield im

In [4]:
def slide_windows(im, step_size, win_size, padding=2):
    for y in range(padding, im.shape[0]-win_size[0]-padding, step_size):
        for x in range(padding, im.shape[1]-win_size[1]-padding, step_size):
            yield (y, x, im[y:y + win_size[0], x:x + win_size[1]])

In [5]:
step_size = 5
win_size = (64, 32)
probability_thresold = 0.9

In [121]:
# img = cv2.imread("dataset/cctv_pos/1_231.png")
img = cv2.imread("sample_5.jpg")
# img = cv2.imread("dataset/human_pos/16.png")

In [107]:
border_type = cv2.BORDER_CONSTANT
border_top = border_bottom = int(win_size[0])
border_left = border_right = int(win_size[1])
def make_border(img):
    border_img = cv2.copyMakeBorder(
        img,
        border_top,
        border_bottom,
        border_left,
        border_right,
        border_type,
        None,
        None
    )
    return border_img

img = make_border(img)

In [122]:
bounding_boxs = np.zeros(5, np.int64)
pyramids = pyramid(img)
for c, pyr in enumerate(pyramids):
    for _, (y, x, patch) in enumerate(slide_windows(pyr, step_size, win_size)):
        if patch.shape[0] != win_size[0] or patch.shape[1] != win_size[1]:
            continue
            
        resized_patch = cv2.resize(patch, WIN_SIZE)
        descriptor = np.transpose(hog_human.compute(resized_patch))
        y_predict = clf.predict(descriptor)
        y_probability = clf.predict_proba(descriptor)
        y_true_probability = y_probability[0][1]
        
        match = y_true_probability > probability_thresold
        
        if match:
            scale_rate = img.shape[0]/pyr.shape[0]
            bounding_boxs = np.vstack((
                bounding_boxs,
                np.array([
                    int(y*scale_rate),
                    int(x*scale_rate),
                    int((y+patch.shape[0])*scale_rate),
                    int((x+patch.shape[1])*scale_rate),
                    int(y_true_probability*1000)]
            )))
        
        pyr_2 = pyr.copy()
        color = (255, 0, 0) if match else (127,127, 127)
        drawed = cv2.rectangle(pyr_2, (x, y), (x + win_size[1], y + win_size[0]), color, 2)
        cv2.putText(drawed, "{:.2f}".format(y_true_probability), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 
                   0.35, (255, 255, 0), 1, cv2.LINE_AA)
        cv2.imshow("slide", drawed)
        cv2.waitKey(500 if match else 1)
bounding_boxs = np.delete(bounding_boxs, (0), axis=0)
cv2.destroyAllWindows()

In [123]:
ratio_prob_scale = 0.8
def nmsf(boxes, overlapThresh):
    if len(boxes) == 0:
        return []

    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    pick = []
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]

    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(
        ratio_prob_scale*((boxes[:,4]/1000-probability_thresold)/(1-probability_thresold))
        +
        (1-ratio_prob_scale)*((y2-y1)/win_size[0])
    )
    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[idxs[:last]]
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))
    return boxes[pick].astype("int")

In [124]:
detected_img = img.copy()
nms_img = img.copy()
nms_bounding_boxs = nmsf(bounding_boxs, 0.3)
for bb in bounding_boxs:
    cv2.rectangle(detected_img, (bb[1], bb[0]), (bb[3], bb[2]), (255, 0, 0), 2)
    cv2.putText(detected_img, str(bb[4]/1000), (bb[1], bb[0]), cv2.FONT_HERSHEY_SIMPLEX, 
                   0.35, (255, 255, 0), 1, cv2.LINE_AA)
cv2.imshow("detected", detected_img)

for nbb in nms_bounding_boxs:
    cv2.rectangle(nms_img, (nbb[1], nbb[0]), (nbb[3], nbb[2]), (255, 0, 0), 2)
    cv2.putText(nms_img, str(nbb[4]/1000), (nbb[1], nbb[0]), cv2.FONT_HERSHEY_SIMPLEX, 
                   0.35, (255, 255, 0), 1, cv2.LINE_AA)
cv2.imshow("after nms", nms_img)

cv2.waitKey()
cv2.destroyAllWindows()